# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [10]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [101]:
pd.crosstab(df['admit'],df['prestige'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [31]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
print dummy_ranks.head(4)

   prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0           0.0           0.0           1.0           0.0
1           0.0           0.0           1.0           0.0
2           1.0           0.0           0.0           0.0
3           0.0           0.0           0.0           1.0


#### 2.2 When modeling our class variables, how many do we need? 



In [13]:
# we would need 4 variables

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [32]:
cols_to_keep = ["admit"]
df_ready = df[cols_to_keep].join(dummy_ranks)
print df_ready.columns
df_ready.head(4)

Index([u'admit', u'prestige_1.0', u'prestige_2.0', u'prestige_3.0',
       u'prestige_4.0'],
      dtype='object')


,admit,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0.0,0.0,1.0,0.0
1,1,0.0,0.0,1.0,0.0
2,1,1.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,1.0


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [34]:
pd.crosstab(df_ready['admit'],df_ready['prestige_1.0'])

prestige_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


In [63]:
33.0/(28.0+33.0)

0.5409836065573771

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [72]:
93.0/(243.0+93.0)

0.2767857142857143

#### 3.3 Calculate the odds ratio

In [73]:
0.2767857142857143/0.5409836065573771

0.5116341991341992

#### 3.4 Write this finding in a sentenance: 

In [ ]:
# the OR is calculated based upon the odds of admission if you attended a tier 1 ranked school divided by the odds of admission if you did not attend a tier 1 ranked school.

#### 3.5 Print the cross tab for prestige_4

In [37]:
pd.crosstab(df_ready['admit'],df_ready['prestige_4.0'])

prestige_4.0,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [46]:
12.0/(55.0+12.0)

0.1791044776119403

In [49]:
114.0/(216.0+114.0)

0.34545454545454546

In [55]:
0.1791044776119403/0.34545454545454546

0.5184603299293008

#### 3.7 Write this finding in a sentence

In [ ]:
# the OR is calculated based upon the odds of admission if you attended a tier 1 ranked school divided by the odds of admission if you did not attend a tier 1 ranked school.

## Part 4. Analysis

In [81]:
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print data.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [82]:
df_ready['intercept'] = 1
df_ready.head(4)

,admit,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,0.0,0.0,1.0,0.0,1
1,1,0.0,0.0,1.0,0.0,1
2,1,1.0,0.0,0.0,0.0,1
3,1,0.0,0.0,0.0,1.0,1


#### 4.1 Set the covariates to a variable called train_cols

In [83]:
train_cols = ['prestige_1.0','prestige_2.0','prestige_3.0',"intercept"]
df_predictors = df_ready[train_cols]
df_predictors.head(4)

,prestige_1.0,prestige_2.0,prestige_3.0,intercept
0,0.0,0.0,1.0,1
1,0.0,0.0,1.0,1
2,1.0,0.0,0.0,1
3,0.0,0.0,0.0,1


#### 4.2 Fit the model

In [84]:
logit = sm.Logit(df_ready['admit'], df_predictors)
results = logit.fit()
print type(results)

Optimization terminated successfully.
         Current function value: 0.593375
         Iterations 5
<class 'statsmodels.discrete.discrete_model.BinaryResultsWrapper'>


#### 4.3 Print the summary results

In [85]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      393
Method:                           MLE   Df Model:                            3
Date:                Tue, 19 Jul 2016   Pseudo R-squ.:                 0.05042
Time:                        15:28:45   Log-Likelihood:                -235.57
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.533e-05
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
prestige_1.0     1.6867      0.409      4.121      0.000         0.885     2.489
prestige_2.0     0.9388      0.362      2.595      0.009         0.230     1.648
prestige_3.0     0.3220      0.385      0.837      0.403        -0.432     1.076
intercept       -1.5224      0.319     -4.778      0.000        -2.147    -0.898
================================================================================
"""

In [87]:
print np.exp(model_coef)

prestige_1.0    5.401786
prestige_2.0    2.557018
prestige_3.0    1.379928
intercept       0.218182
dtype: float64


In [86]:
model_coef = results.params
print model_coef

prestige_1.0    1.686730
prestige_2.0    0.938842
prestige_3.0    0.322032
intercept      -1.522427
dtype: float64


#### 4.5 Interpret the OR of Prestige_2

In [ ]:
# students admitted to tier 1 ranked school have a 79% higher probability of admission versus students admitted to a tier 2 ranked school.  This is the ratio of 1.686730 for prestige_1 to .938842 for prestige_2.

#### 4.6 Interpret the OR of GPA

In [ ]:
# OR of .38 suggests students have a 38% higher probability of admission between tier 1 and tier 2 schools based upon GPA.

## Part 5: Predicted probablities


In [114]:
def cartesian(arrays, out=None):
    
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [115]:
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
combos = pd.DataFrame(cartesian([gres, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]


#### 5.1 Recreate the dummy variables

In [103]:
prestige_test_set = np.random.randint(1,5,50)
print prestige_test_set
df_test = pd.DataFrame({'prestige':prestige_test_set, 'intercept': 1})

[4 4 1 3 1 3 4 1 1 4 3 2 1 3 3 2 3 1 4 3 4 2 4 1 3 2 2 2 4 2 1 2 3 3 2 4 2
 3 2 3 1 3 4 4 3 2 1 1 4 1]


In [104]:
df_test.describe()

,intercept,prestige
count,50.0,50.000000
mean,1.0,2.520000
std,0.0,1.110984
min,1.0,1.000000
25%,1.0,2.000000
50%,1.0,3.000000
75%,1.0,3.000000
max,1.0,4.000000


In [105]:
dummy_ranks_test = pd.get_dummies(df_test['prestige'], prefix='Prestige')
dummy_ranks_test.columns

Index([u'Prestige_1', u'Prestige_2', u'Prestige_3', u'Prestige_4'], dtype='object')

In [106]:
test_cols_to_keep = ["Prestige_1",'Prestige_2','Prestige_3',"intercept"] 
df_for_pred = df_test.join(dummy_ranks_test)[test_cols_to_keep]
df_for_pred.head(3)

,Prestige_1,Prestige_2,Prestige_3,intercept
0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,1
2,1.0,0.0,0.0,1


#### 5.2 Make predictions on the enumerated dataset

In [107]:
df_for_pred['admit_pred'] = results.predict(df_for_pred)
print df_for_pred.tail()
len(df_for_pred)

    Prestige_1  Prestige_2  Prestige_3  intercept  admit_pred
45         0.0         1.0         0.0          1    0.358108
46         1.0         0.0         0.0          1    0.540984
47         1.0         0.0         0.0          1    0.540984
48         0.0         0.0         0.0          1    0.179104
49         1.0         0.0         0.0          1    0.540984


50

#### 5.3 Interpret findings for the last 4 observations

In [ ]:
# the predicted probability of admission to graduate school is approximately 54% is student is from a tier 1 school.